In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
#פונקציה לקבלת תאריך מהמשתמש, מוכנה 
import pandas as pd
from datetime import datetime, timedelta

### קריאה

### אימון המודל בהתחלה

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler

# Replace 'your_file.csv' with the path to your CSV file
file_path = 'pedestrian_data.csv'

# Read the CSV file into a DataFrame
data = pd.read_csv(file_path)



#יוצר דאטה פריים של כל הצירופים האפשרים
unique_combinations = data[['Start', 'End']].drop_duplicates()


unique_combinations


# הגדרת X ו Y למודל 
feature_columns = ['Hour_00:00', 'Hour_01:00', 'Hour_02:00', 'Hour_03:00',
                   'Hour_04:00', 'Hour_05:00', 'Hour_06:00', 'Hour_07:00', 'Hour_08:00',
                   'Hour_09:00', 'Hour_10:00', 'Hour_11:00', 'Hour_12:00', 'Hour_13:00',
                   'Hour_14:00', 'Hour_15:00', 'Hour_16:00', 'Hour_17:00', 'Hour_18:00',
                   'Hour_19:00', 'Hour_20:00', 'Hour_21:00', 'Hour_22:00', 'Hour_23:00',
                   'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                   'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
                   'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
                   'Wednesday', 'Autumn', 'Spring', 'Summer', 'Winter']

# Define the target column
target_column = 'Travel time'

# Initialize results storage
results = []
num = 1
for index, row in unique_combinations.iterrows():
    start, end = row['Start'], row['End']
    filtered_data = data[(data['Start'] == start) & (data['End'] == end)]
    filtered_data = filtered_data.drop(['Start', 'End'], axis=1)

    if filtered_data.empty:
        continue

    # Split data into training and test sets
    X = filtered_data[feature_columns]
    y = filtered_data[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale the features
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Hyperparameter tuning for XGBRegressor
    params = {
        'learning_rate': [0.01, 0.1],
        'n_estimators': [100, 200],
        'max_depth': [3, 5],
        'subsample': [0.8],
        'colsample_bytree': [0.8],
        'min_child_weight': [1, 3]
    }

    grid = GridSearchCV(XGBRegressor(random_state=42), param_grid=params, scoring='r2', cv=5, verbose=1)
    grid.fit(X_train_scaled, y_train)

    best_model = grid.best_estimator_
    print(f'Best parameters for Start: {start}, End: {end} - {grid.best_params_}')
    print('num of run :',num)
    num=num+1
    

    # Evaluate the best model
    y_pred = best_model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Store the results
    results.append({
        'Start': start,
        'End': end,
        'R-squared Score': r2,
        "model":grid
            })


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters for Start: Holmium, End: Promethium - {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 0.8}
num of run : 1
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters for Start: Neodymium, End: Dysprosium - {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.8}
num of run : 2
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters for Start: Thorium, End: Polonium - {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200, 'subsample': 0.8}
num of run : 3
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best parameters for Start: Technetium, End: Indium - {'colsample_bytree': 0.8, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 200, 'subsample

### שמירה לקובץ חיצוני

In [ ]:
import pickle

# Save the results variable to a file
with open('results.pkl', 'wb') as file:
    pickle.dump(results, file)

In [ ]:
import joblib

#שמירה המודל
joblib.dump(best_model, 'best_xgb_model.pkl')

### בדיקה חיזוי

In [ ]:
#בדיקה של הפרדיקט
Y = (1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1)
y = np.array(Y).reshape(1, -1)  # משנה את הצורה למערך דו-מימדי עם 1 דוגמה ו-47 תכונות
# עכשיו y הוא מערך עם צורה (1, 47)
len(Y)
y.shape
results[45]["model"].predict(y)
results[0]["model"]

# המשך קוד

# מעבר מסטרינג לוקטור של 1/47

In [53]:
# Convert results to DataFrame and save to CSV
results_df = pd.DataFrame(results)

In [16]:
#פונקציה לקבלת תאריך מהמשתמש, מוכנה 

def process_on_Date(start_datetime):
    
    #קובץ אימון
    #datetime_obj = datetime.strptime(start_datetime, '%d/%m/%Y %H:%M')
    datetime_obj = datetime.strptime(start_datetime, '%Y-%m-%d %H:%M:%S')

    
    # Split into date and time
    date_str = datetime_obj.strftime('%Y-%m-%d')
    time_str = datetime_obj.strftime('%H:%M:%S')
    
    # Round each time value to the nearest hour
    time_obj = datetime.strptime(time_str, '%H:%M:%S')
    rounded_time = (time_obj + timedelta(minutes=30)).replace(minute=0).strftime('%H:%M') if time_obj.minute < 30 else (time_obj + timedelta(hours=1)).replace(minute=0).strftime('%H:%M')
    
    # Create a DataFrame to hold this data
    data = pd.DataFrame({
        'Date': [date_str],
        'Time': [time_str],
        'new_time': [rounded_time]
    })
       

    # Convert 'Date' to datetime and extract month
    data['Date'] = pd.to_datetime(data['Date'])
    data['Date month'] = data['Date'].dt.month
    
    # Create dummies for time, month, and day of the week
    dummies_time = pd.get_dummies(data['new_time'], prefix='Hour', drop_first=True)  # drop_first to avoid multicollinearity
    dummies_month = pd.get_dummies(data['Date month'], prefix='month', drop_first=True)  # drop_first to avoid multicollinearity
    

    data['Day_of_Week'] = data['Date'].dt.day_name()
    dummies_day = pd.get_dummies(data['Day_of_Week'], prefix='Day', drop_first=True)  # drop_first to avoid multicollinearity
    
    # Determine the season
    data['season'] = ''
    data.loc[data['Date month'].isin([12, 1, 2]), 'season'] = 'Winter'
    data.loc[data['Date month'].isin([3, 4, 5]), 'season'] = 'Spring'
    data.loc[data['Date month'].isin([6, 7, 8]), 'season'] = 'Summer'
    data.loc[data['Date month'].isin([9, 10, 11]), 'season'] = 'Autumn'
    dummies_season = pd.get_dummies(data['season'], drop_first=True)  # drop_first to avoid multicollinearity
    
    # Create empty DataFrames for all months, hours, days of the week, and seasons
    all_months = pd.DataFrame(columns=[f'month_{i}' for i in range(1, 13)])
    all_hours = pd.DataFrame(columns=[f'Hour_{i:02d}' for i in range(0, 24)])
    all_days = pd.DataFrame(columns=[f'Day_{day}' for day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']])
    all_seasons = pd.DataFrame(columns=['Spring', 'Summer', 'Autumn', 'Winter'])
    
    # Concatenate with existing dummy variables to ensure all months, hours, days of the week, and seasons are represented
    dummies_month = pd.concat([dummies_month, all_months], axis=1)
    dummies_time = pd.concat([dummies_time, all_hours], axis=1)
    dummies_day = pd.concat([dummies_day, all_days], axis=1)
    dummies_season = pd.concat([dummies_season, all_seasons], axis=1)
    
    # Concatenate all dummies with the original data
    data = pd.concat([data, dummies_time, dummies_month, dummies_day, dummies_season], axis=1)
    
    # Convert specific columns to integer type, fill NaNs with 0
    data[dummies_time.columns] = data[dummies_time.columns].fillna(0).astype(int)
    data[dummies_month.columns] = data[dummies_month.columns].fillna(0).astype(int)
    data[dummies_day.columns] = data[dummies_day.columns].fillna(0).astype(int)
    data[dummies_season.columns] = data[dummies_season.columns].fillna(0).astype(int)
    

    
    # Set the dummy variables for the provided datetime string to 1
    provided_month = datetime_obj.month
    provided_hour = datetime_obj.hour
    provided_day = datetime_obj.strftime('%A')
    provided_season = 'Winter' if provided_month in [12, 1, 2] else \
                     'Spring' if provided_month in [3, 4, 5] else \
                     'Summer' if provided_month in [6, 7, 8] else 'Autumn'
    
    data[f'month_{provided_month}'] = 1
    data[f'Hour_{provided_hour:02d}'] = 1
    data[f'Day_{provided_day}'] = 1
    data[provided_season] = 1
    
    
    # Drop unnecessary columns
    column_to_drop = ['Date', 'Time', 'Day_of_Week', 'Date month', 'season','new_time']
    data.drop(columns=column_to_drop, axis=1, inplace=True)
        
    
    y = data
    y = np.array(y).reshape(1, -1)   # מעביר לווקטור של 47   
    


    return y



### יצירה של משקלים לקשתות

In [18]:
def Make_pred_matrix(datime):
    # משתנה בהתאם לסוג קובץ
    file_path = 'pedestrian_data.csv'    
    #הכנה כללית
    # Read the CSV file into a DataFrame
    data = pd.read_csv(file_path)
    unique_combinations = data[['Start', 'End']].drop_duplicates()
    
    
    y = process_on_Date(datime)

    # Initialize an empty list to store predictions
    all_predictions = []
    

    

    # לולאה על כל קומבינציה ייחודית
    for index, row in results_df.iterrows():
        start_point = row['Start']
        end_point = row['End']

        # קבלת ערך Y באמצעות הפונקציה המותאמת אישית
        # ביצוע הניבוי באמצעות המודל במילון results
        prediction = results[index]["model"].predict(y)

        # Create a dictionary for the current prediction
        prediction_dict = {
            'Start': start_point,
            'End': end_point,
            'prediction': prediction
        }

        # Add the prediction dictionary to the list
        all_predictions.append(prediction_dict)

    # Create a DataFrame from the list of prediction dictionaries
    predictions_df = pd.DataFrame(all_predictions)

    # Save the DataFrame to a CSV file
    predictions_df.to_csv('predictions.csv', index=False)

    return predictions_df

### דייקסטרה

In [20]:
import networkx as nx

def dijkstra_with_nx(graph, start, end):
    # Create a directed graph
    G = nx.DiGraph()
    
    graph['prediction'] = graph['prediction'].clip(lower=0)

    # Add edges from DataFrame
    for _, row in graph.iterrows():
        G.add_edge(row['Start'], row['End'], weight=row['prediction'])

    # Compute shortest path using Dijkstra's algorithm
    shortest_path = nx.shortest_path(G, source=start, target=end, weight='weight')

    # Compute shortest distance
    shortest_distance = nx.shortest_path_length(G, source=start, target=end, weight='weight')

    return shortest_path, shortest_distance




### טעינה קבצים לבחינה

In [ ]:
# Replace 'your_file.csv' with the path to your CSV file
file_path = '/kaggle/input/data-pswd/DATA pswd.csv'

# Read the CSV file into a DataFrame
og_df = pd.read_csv(file_path)
og_df.head()

In [ ]:
# Replace 'your_file.csv' with the path to your CSV file
file_path = '/kaggle/input/final-test/FINAL_TEST.xlsx'

# Read the CSV file into a DataFrame
test_data_df = pd.read_excel(file_path)
test_data_df.head(100)

 # פונקציה סופית  

In [22]:
def final_fun_for_pred(data_for_test_df):
    pred_path_list = []  
    pred_distance_list = []  
    i=1
    for index, row in data_for_test_df.iterrows():
        start_datetime = str(row['Date'])  # Convert the datetime to a string
        predictions_graph_df = Make_pred_matrix(start_datetime)
        start_point = row['Start']
        end_point = row['End']
        pred_path, pred_distance = dijkstra_with_nx(predictions_graph_df, start_point, end_point)
        pred_path_list.append(pred_path)
        pred_distance_list.append(pred_distance)
        
        print('num of run :',i)
        i=i+1

    data_for_test_df['Pred_Path'] = pred_path_list
    data_for_test_df['Fastest_Pred_Distance'] = pred_distance_list
    

    
    
    
    
    return data_for_test_df

In [24]:
import warnings

# התעלמות מכל ההתראות
warnings.filterwarnings("ignore")

In [ ]:
test_result_df = final_fun_for_pred(test_data_df)

In [ ]:
test_result_df.to_csv('test_result_df.csv', index=False)

# הכנה לקראת הגוי

In [45]:
source = input("Enter the source: ")
destination = input("Enter the destination: ")    

Enter the source:  Thallium
Enter the destination:  Argon


In [47]:
from datetime import datetime
# קבלת תאריך מהמשתמש כטקסט
date_input = input("Enter the date (YYYY-MM-DD): ")
# המרת הקלט לאובייקט datetime
datetime_obj = datetime.strptime(date_input, '%Y-%m-%d')
# המרת אובייקט datetime למחרוזת בפורמט 'YYYY-MM-DD'
date_str = datetime_obj.strftime('%Y-%m-%d')



Enter the date (YYYY-MM-DD):  2014-02-02


In [49]:
time = input("Enter the time (HH:MM): ")
time = datetime_obj.strftime('%H:%M:%S')

Enter the time (HH:MM):  02:02:02


In [83]:
x,y = final_fun_for_GUI(source,destination,date_str,time)

In [104]:
type(y[0])

numpy.float32

In [106]:
numstr  = str(y[0])

In [108]:
numstr

'2599.9043'

# GUI

In [34]:
ListOfCities = [
    "Gadolinium", "Holmium", "Thallium", "Neodymium", "Thorium", "Technetium",
    "Ytterbium", "Tantalum", "Vanadium", "Neon", "Protactinium", "Erbium", "Xenon",
    "Francium", "Promethium", "Astatine", "Argon", "Lanthanum", "Mercury", "Lithium",
    "Sulfur", "Zirconium", "Radon", "Potassium", "Germanium", "Cerium", "Hydrogen",
    "Strontium", "Manganese", "Iron", "Krypton", "Arsenic", "Rhenium", "Antimony",
    "Fluorine", "Nitrogen", "Actinium", "Neptunium", "Oxygen", "Ruthenium", "Aluminum",
    "Helium", "Chromium", "Praseodymium", "Cesium", "Tellurium", "Lead", "Samarium",
    "Calcium", "Europium", "Platinum", "Radium", "Tin", "Carbon", "Tungsten", "Selenium",
    "Scandium", "Iridium", "Uranium", "Beryllium", "Americium", "Phosphorus", "Gallium",
    "Iodine", "Silicon", "Zinc", "Yttrium", "Thulium", "Copper", "Rhodium", "Curium",
    "Osmium", "Barium", "Molybdenum", "Cobalt", "Magnesium", "Silver", "Bismuth",
    "Polonium", "Chlorine", "Indium", "Plutonium", "Niobium", "Boron", "Rubidium",
    "Hafnium", "Bromine", "Nickel", "Lutetium", "Dysprosium", "Gold", "Terbium",
    "Palladium", "Sodium", "Cadmium","Titanium"]


In [29]:
def final_fun_for_GUI(source,destination,date,time):
    
    start_datetime = date + " "+ time
    predictions_graph_df = Make_pred_matrix(start_datetime)
    start_point = source
    end_point = destination
    pred_path, pred_time = dijkstra_with_nx(predictions_graph_df, start_point, end_point)
    
    return pred_path, pred_time



In [126]:
import customtkinter as ctk
from tkinter import messagebox
from tkcalendar import DateEntry
import tkinter as tk

def show_custom_message(message1, message2, bg_color="blue", text_color="#333"):
    custom_message_window = tk.Toplevel(window)  # יצירת חלון פרטי
    custom_message_window.title("Route Information")  # כותרת לחלון

    frame = tk.Frame(custom_message_window, bg="#f0f0f0", padx=20, pady=20)
    frame.pack(expand=True, fill="both")

    # יצירת תווית עם הודעה הראשונה
    label1 = tk.Label(frame, text=message1, font=("Arial", 14), padx=10, pady=5, bg="#f0f0f0", fg="#333")
    label1.pack()

    # יצירת תווית עם הודעה השנייה
    label2 = tk.Label(frame, text=message2, font=("Arial", 14), padx=10, pady=5, bg="#f0f0f0", fg="#333")
    label2.pack()

    # שינוי גודל החלון
    custom_message_window.geometry("1100x200")

    # הצגת החלון במרכז המסך
    custom_message_window.update_idletasks()
    width = custom_message_window.winfo_width()
    height = custom_message_window.winfo_height()
    x = (custom_message_window.winfo_screenwidth() // 2) - (width // 2)
    y = (custom_message_window.winfo_screenheight() // 2) - (height // 2)
    custom_message_window.geometry('{}x{}+{}+{}'.format(width, height, x, y))

    # אפשרות להצגת חלון עם סגנון גבוה
    custom_message_window.attributes("-topmost", True)


# פונקציה לחישוב הנסיעה
def calculate_journey():
    starting_station = starting_station_combo.get()
    destination_station = destination_station_combo.get()
    #date = date_entry.get()
    date = departure_date_entry.get()
    datetime_obj = datetime.strptime(date, '%Y-%m-%d')
    # המרת אובייקט datetime למחרוזת בפורמט 'YYYY-MM-DD'
    date_str = datetime_obj.strftime('%Y-%m-%d')
    departure_time = departure_time_entry.get()

    # אם אחד מהפרטים חסרים
    if starting_station == "" or destination_station == "" or date == "" or departure_time == "":
        messagebox.showerror("שגיאה", "יש למלא את כל השדות")
    else:
        
        pred_path, pred_time = final_fun_for_GUI(starting_station,destination_station,date,departure_time)
        route = "->".join(pred_path)
        timestrnumber = str(pred_time[0])
        message1 = f"The quickest route from {starting_station} to {destination_station} on {date} at {departure_time} is : \n" + route + "\n"
        message2 = "Route time : " + timestrnumber + "s" 
        show_custom_message(message1, message2)

# יצירת חלון
window = ctk.CTk()
window.title("Travel Booking System")

# קביעת גודל החלון
window.geometry("300x400")

# קביעת תצורת הרשת
for i in range(9):
    window.grid_rowconfigure(i, weight=1)
window.grid_columnconfigure(0, weight=1)

# תווית ובחירת תחנת מוצא
starting_station_label = ctk.CTkLabel(window, text="Starting station:")
starting_station_label.grid(row=0, column=0, padx=10, pady=(20, 5), sticky="e")

starting_station_var = ctk.StringVar()
starting_station_combo = ctk.CTkComboBox(window, variable=starting_station_var, values=ListOfCities)
starting_station_combo.grid(row=1, column=0, padx=10, pady=(0, 5), sticky="we")

# תווית ובחירת תחנת יעד
destination_station_label = ctk.CTkLabel(window, text="Destination station:")
destination_station_label.grid(row=2, column=0, padx=10, pady=(20, 5), sticky="e")

destination_station_var = ctk.StringVar()
destination_station_combo = ctk.CTkComboBox(window, variable=destination_station_var, values=ListOfCities)
destination_station_combo.grid(row=3, column=0, padx=10, pady=(0, 5), sticky="we")

# תווית ובחירת תאריך
#date_label = ctk.CTkLabel(window, text="Date:")
date_label = ctk.CTkLabel(window, text="Date:")
date_label.grid(row=4, column=0, padx=10, pady=(20, 5), sticky="e")
#date_entry = DateEntry(window, date_pattern='yy/mm/dd')
departure_date_var = ctk.StringVar()
departure_date_entry = ctk.CTkEntry(window, textvariable=departure_date_var)
departure_date_entry.grid(row=5, column=0, padx=10, pady=(0, 5), sticky="we")

# תווית ובחירת זמן תחילת נסיעה
departure_time_label = ctk.CTkLabel(window, text="Departure time (hh:mm:ss):")
departure_time_label.grid(row=6, column=0, padx=10, pady=(20, 5), sticky="e")

departure_time_var = ctk.StringVar()
departure_time_entry = ctk.CTkEntry(window, textvariable=departure_time_var)
departure_time_entry.grid(row=7, column=0, padx=10, pady=(0, 5), sticky="we")

# כפתור לחישוב הנסיעה
calculate_button = ctk.CTkButton(window, text="Calculate Journey", command=calculate_journey)
calculate_button.grid(row=8, column=0, padx=10, pady=10, sticky="we", columnspan=2)






In [ ]:
window.mainloop()